In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

Ensure that data saved is still present. Then load data set

In [2]:
!ls

cloudclassifier.npy  cloudtrainingdata.csv  x_test.npy	 y_test.npy
clouddata.npz.zip    testset.npy	    x_train.npy  y_train.npy


In [4]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')


In [5]:
#initializing placeholders and parameters allows TensorFlow to build a dataflow graph
def placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, shape=(n_x, None), name=None)
    Y = tf.placeholder(tf.float32, shape=(n_y, None), name=None)
    
    return X, Y

def init_parameters():
    
    W1 = tf.get_variable("W1", [1024,13358], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [1024,1], initializer = tf.zeros_initializer())
    W2 = tf.get_variable("W2", [512,1024], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [512,1], initializer = tf.zeros_initializer())
    W3 = tf.get_variable("W3", [64,512], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [64,1], initializer = tf.zeros_initializer())
    W4 = tf.get_variable("W4", [4,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b4 = tf.get_variable("b4", [4,1], initializer = tf.zeros_initializer())

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4}
    
    return parameters

#data is fed through a neural network via forward propagation in order to form a prediction
def forward_prop(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    
    Z1 = tf.add(tf.matmul(W1,X), b1)                      
    A1 = tf.nn.relu(Z1)                                   
    Z2 = tf.add(tf.matmul(W2,A1), b2)                   
    A2 = tf.nn.relu(Z2)                                   
    Z3 = tf.add(tf.matmul(W3,A2), b3)  
    A3 = tf.nn.relu(Z3)
    Z4 = tf.add(tf.matmul(W4,A3), b4)
    
    return Z4

#measure probability error in predictions in order to alter and optimize weights
def compute_cost(Z4, Y):
   
    logits = tf.transpose(Z4)
    labels = tf.transpose(Y)
   
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    
    return cost

In [38]:
def model(X_train, Y_train, learning_rate = 0.0001,
          num_epochs = 200, print_cost = True):
    
    ops.reset_default_graph()                         
    (n_x, m) = X_train.shape                          
    n_y = Y_train.shape[0]                            
    costs = []                                        
    
    #put the above functions together, and optimize weights' value and minimize cost via Adam optimizer and save trained parameters
    X, Y = placeholders(n_x, n_y)
    parameters = init_parameters()
    Z4 = forward_prop(X, parameters)
    cost = compute_cost(Z4, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:

        sess.run(init)
        for epoch in range(num_epochs):
            _ , epoch_cost = sess.run([optimizer, cost], feed_dict={X: X_train, Y: Y_train})
            if print_cost == True and epoch % 20 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        correct_prediction = tf.equal(tf.argmax(Z4), tf.argmax(Y))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        
        save_path = saver.save(sess, "params.ckpt",global_step=epoch)
        
        return parameters

In [39]:
parameters = model(x_train, y_train)

Cost after epoch 0: 1.549277
Cost after epoch 20: 0.144776
Cost after epoch 40: 0.061817
Cost after epoch 60: 0.039667
Cost after epoch 80: 0.028857
Cost after epoch 100: 0.021995
Cost after epoch 120: 0.017139
Cost after epoch 140: 0.013576
Cost after epoch 160: 0.010895
Cost after epoch 180: 0.008858
Parameters have been trained!
Train Accuracy: 1.0


### Trained weights of network saved for further use. Check for addition of new files using ls command

In [24]:
!ls

checkpoint	       params.ckpt-4.data-00000-of-00001  x_test.npy
cloudclassifier.npy    params.ckpt-4.index		  x_train.npy
clouddata.npz.zip      params.ckpt-4.meta		  y_test.npy
cloudtrainingdata.csv  testset.npy			  y_train.npy
